In [39]:
from util import *
from aocd import data, submit

In [2]:
sample_data = """sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew"""

In [25]:
from dataclasses import dataclass
import hashlib

Coordinate system:
```
  -1,-1  0, -1
-1,0  0,0  1,0
    0,1  1,1
 ```

In [133]:
EAST = 'e'
WEST = 'w'
N_EAST = 'ne'
N_WEST = 'nw'
S_EAST = 'se'
S_WEST = 'sw'

direction_delta_map = {
    # even/odd row deltas
    EAST: [(1, 0)],
    WEST: [(-1, 0)],
    N_EAST: [(1, -1), (0, -1)],
    N_WEST: [(0, -1), (-1, -1)],
    S_EAST: [(1, 1), (0, 1)],
    S_WEST: [(0, 1), (-1, 1)],
}

@dataclass
class Hex():
    col: int = 0
    row: int = 0
        
    def translate(self, op) -> 'Hex':
        row_type = -1 * (self.row % 2) # map to 0 for even, -1 for odd
        d_c, d_r = direction_delta_map[op][row_type]
        return Hex(col = self.col + d_c, row = self.row + d_r)
        
    def __hash__(self):
        return hash((self.col, self.row))
    
    def neighbors(self) -> List['Hex']:
        return mapl(lambda op: self.translate(op), direction_delta_map.keys())
    

In [134]:
%load_ext ipython_unittest

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


In [161]:
%%unittest_testcase
def test_even_ne(self):
    updated = Hex(3, 2).translate('ne')
    self.assertEqual(updated, Hex(4, 1))

def test_even_nw(self):
    updated = Hex(3, 2).translate('nw')
    self.assertEqual(updated, Hex(3, 1))
    
def test_odd_ne(self):    
    updated = Hex(3, 1).translate('ne')
    self.assertEqual(updated, Hex(3,0))

def test_odd_nw(self):
    updated = Hex(3, 1).translate('nw')
    self.assertEqual(updated, Hex(2, 0))
    
def test_neighbors(self):
    neighbors = Hex(3, 1).neighbors()
    self.assertEqual(len(set(neighbors)), 6)

Success

.....
----------------------------------------------------------------------
Ran 5 tests in 0.000s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

In [136]:
import re

In [137]:
def parse_data(data) -> List[List[str]]:
    return mapl(lambda row: re.findall('se|sw|ne|nw|e|w', row), data.splitlines())

In [138]:
from functools import reduce

In [139]:
def part_1(data):
    instructions = parse_data(data)
    tile_map = defaultdict(lambda : False)
    for instruction in instructions:
        target_tile = reduce(lambda h, op: h.translate(op), instruction, Hex())
        # print(f"Flipping hex {target_tile}, prev state: {tile_map[target_tile]}")
        tile_map[target_tile] = not tile_map[target_tile] 
    return tile_map

In [140]:
tile_map = part_1(data)

In [141]:
num_black = sum(tile_map.values())

In [142]:
num_black

232

In [120]:
# submit(num_black)

answer a: None
submitting for part a


That's the right answer!  You are one gold star closer to saving your vacation. [Continue to Part Two]


<Response [200]>

## Part 2

In [176]:
def count_black_neighbors(tile, state):
    return sum(mapl(lambda cand: state[cand], tile.neighbors()))

In [182]:
def apply_step(state):
    new_state = defaultdict(lambda : False)
    black_tiles = dict(filterl(lambda item: item[1], state.items()))
    white_neighbors = set(chain.from_iterable(mapl(lambda tile: tile.neighbors(), black_tiles.keys()))) - set(black_tiles.keys())
    
    for black_tile in black_tiles.keys():
        count = count_black_neighbors(black_tile, state)
        # print(f"{black_tile}: {count}")
        if count in (1, 2):
            # stays black
            new_state[black_tile] = True
    
    for white_tile in white_neighbors:
        count = count_black_neighbors(white_tile, state)
        # print(f"{white_tile}: {count}")
        if count == 2:
            new_state[white_tile] = True
    
    return new_state

In [186]:
state = part_1(data)
for i in range(100):
    state = apply_step(state)
    #print(f"{i}: {sum(state.values())}")

In [187]:
result = sum(state.values())
result

3519

In [188]:
submit(result)

answer a: 232
submitting for part b (part a is already completed)


That's the right answer!  You are one gold star closer to saving your vacation.You have completed Day 24! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


<Response [200]>